In [1]:
# Run the script with functions to make sure to import the latest saved version
%run Funzioni_Generali.py
%run Funzioni_Financial_Model.py
%run Funzioni_Energy_Model.py
%run Funzioni_Demand_Side_Management.py

In [2]:
from simple_colors import *

In [3]:
# Import single functions

# general functions
from Funzioni_Generali import (
                              #  check_file_status, province_to_region, 
                               generate_calendar, 
                              #  get_calendar, 
                              generate_users_yml, generate_plant_yml, membership_matrix, plant_operation_matrix, 
                               load_profile_all_users, 
                               edit_incentive_repartition_scheme, save_simulation_results
                               )

# functions for energy simulation
from Funzioni_Energy_Model import (
                                   simulate_configuration_productivity, 
                                   CACER_energy_flows, 
                                   CACER_shared_energy, 
                                #    CACER_injected_energy_optimizer
                                   )

# functions for financial simulation
from Funzioni_Financial_Model import(
                                    create_users_bill, 
                                    add_to_recap_yml, 
                                    RID_calculation, 
                                    aggregate_CACER_bills, aggregate_CACER_RID, 
                                    FM_initialization, 
                                    incentives, 
                                    cash_flows_for_all_plants, cash_flows_for_all_users, 
                                    aggregate_FM, 
                                    organize_simulation_results_for_reporting
                                    )

# functions DSM simulation
from Funzioni_Demand_Side_Management import (create_emulated_users, 
                                            #  create_optimal_appliance_start_time_dictionary
                                             )

---

## **External inputs to set before to run the simulation**

Before to run the simulation be sure to have set correctly the following input files:
        
- config.yml [[link yaml](config.yml)]

- users CACER.xlsx [[link excel](users%20CACER.xlsx)]

---

## **Initializing CACER configuration**

In this part, all the input files needed for the simulation are created.

### **1. Generating the calendar file**

A csv file is created with the following data:
- **datetime**, in format [%Y/%M/%D %h:%m:%s];

- **day week**, [1 : monday, 2 : thuesday, etc.];
- **holiday**, [True : holiday, False : not holiday];
- **fascia**, [1, 2, 3];
- **day_flag**, [monday, thuesday, etc.].

The csv file has the following structure:

| datetime  (type DatetimeIndex)| day_week (type int)| holiday (type boolean)| fascia (type int)| day_flag (type string)|      
| :----:| :----:| :----:| :----:| :----:|
| 01/01/2025  00:00:00| 2| True| 3| Sunday| 
| 01/01/2025  00:15:00| 2| True| 3| Sunday| 
| ...| ...| ...| ...| ...| 

📂  *files/general/calendar.csv*

In [ ]:
print(blue("\n1. Get calendar:", ['bold', 'underlined']), '\n')
generate_calendar()

---

### **2. Generating the yaml and csv files with all the inputs of the CACER configuration**

The following files are created:
- generate_users_yml() 

        → files/recap.yml 

        → files/registry_user_types.yml 

        → files/registry_users.yml

        → files/registry_users.csv

- generate_plant_yml() 

        → files/registry_plant.yml

        → files/registry_plant.csv

- membership_matrix() 

        → files/membership_matrix.csv

- plant_operation_matrix() 

        → files/plant_operation_matrix.xlsx


In [ ]:
print(blue("\n2. Generate parameters in yaml file from xlsx file:", ['bold', 'underlined']), '\n')
generate_users_yml(base=10)
generate_plant_yml() 
membership_matrix()
plant_operation_matrix()

---

### **3. Generating the emulated load profile of the users**

A csv file is created with the following data:
- **datetime**, in format [%Y/%M/%D %h:%m:%s];

- quarterly electrical load of each individual user [kWh/15min].

The csv file has the following structure:

| datetime  (type DatetimeIndex)| user_1 (type float)| user_2 (type float)| ...|      
| :----:| :----:| :----: | :----: |
| 01/01/2025  00:00:00| 0.109| 0.109| ...| 
| 01/01/2025  00:15:00| 0.098| 0.098| ...| 
| ...| ...| ...| ...| 

**emultated users only*

📂 *files/input/emulated_load_profile.csv*

In [ ]:
print(blue("\n3. Create load profile for emulated users:", ['bold', 'underlined']), '\n')

flag_last_dict = False # if false we create the appliance start time dictionary; if true we import the last one created (default: False)
flag_optDSM = False # if true we simulate the optimized DSM case (default: False)
flag_all_appliance = True # if true we use all appliance for the load profile emulation (default: True)
flag_daily_activation = True # if false we dont'use a daily usage activation for some specified appliances (only washing machine at the moment, default: True)
flag_multi_use = True # if true we activate the possibility to have multiple activations for the selected appliances during the day (default: True)

create_emulated_users(flag_last_dict, flag_optDSM, flag_all_appliance, flag_daily_activation, flag_multi_use)

---

### **4. Generating all load profile of the users (domestic users, commercial users, industrial users, etc.)**

A csv file is created with the following data:
- **datetime**, in format [%Y/%M/%D %h:%m:%s];

- quarterly electrical load of each individual user [kWh/15min].

The csv file has the following structure (**carichi.csv**):

| datetime  (type DatetimeIndex)| user_1 (type float)| user_2 (type float)| ...|      
| :----:| :----:| :----: | :----: |
| 01/01/2025  00:00:00| 0.109| 0.109| ...| 
| 01/01/2025  00:15:00| 0.098| 0.098| ...| 
| ...| ...| ...| ...| 

**all users*

📂 *files/energy/carichi.csv*

In [ ]:
print(blue("\n4. Generate load profile for user types added:", ['bold', 'underlined']), '\n')
load_profile_all_users()

---

### **5. Generating productivity of the photovoltaic generators**

A csv file is created with the following data:
- **datetime**, in format [%Y/%M/%D %h:%m:%s];

- quarterly electrical productivity of each individual user [kWh/15min].

The csv file has the following structure:

| datetime  (type DatetimeIndex)| user_1 (type float)| user_2 (type float)| ...|      
| :----:| :----:| :----: | :----: |
| 01/01/2025  00:00:00| 0.109| 0.109| ...| 
| 01/01/2025  00:15:00| 0.098| 0.098| ...| 
| ...| ...| ...| ...| 

**all users*

📂 *files/output_gen_pv.csv*

In [ ]:
print(blue("\n5. Generate production profile for user types added:", ['bold', 'underlined']))
simulate_configuration_productivity()

---

### **6. Generating all energy flows of the single users**

A csv file is created <u>for each users</u> with the following data:
- **datetime**, in format [%Y/%M/%D %h:%m:%s];

- **Eprel**, quarterly withdrawn energy [kWh/15min];
- **Eut**, quarterly consumed energy [kWh/15min];
- **Eimm**, quarterly injected energy [kWh/15min];
- **Eprod**, quarterly produced energy [kWh/15min];
- **Eperdite**, quarterly lost energy [kWh/15min];
- **Eaut**, quarterly self-consumed energy [kWh/15min];
- **Eaut_PV**, quarterly self-consumed energy from photovoltaic generator [kWh/15min];
- **Eaut_batt**, quarterly self-consumed energy from battery [kWh/15min].

The csv files have the following structure:

| datetime  (type DatetimeIndex)| Eprel (type float)| Eut (type float)| Eimm (type float)| Eprod (type float)| Eperdite (type float)| Eaut (type float)| Eaut_PV (type float)| Eaut (type float)|      
|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|
| 01/01/2025  00:00:00| 0.0899| 0.0899| 0.0899| 0.0899| 0.0899| 0.0899| 0.0899| 0.0899|  
| 01/01/2025  00:15:00| 0.0909| 0.0909| 0.0909| 0.0909| 0.0909| 0.0909| 0.0909| 0.0909| 
| ...| ...| ...| ...| ...| ...| ...| ...| ...| 

📂 *files/energy/profili_utenti/*

In [ ]:
print(blue("\n6. Generate all CER energy flows:", ['bold', 'underlined']), '\n')
CACER_energy_flows()

---
---

## **Simulate CACER configuration**

In this part, all economic cash flows (aggregated for the project, aggregated for the CACER configuratione and for each user types) are simulated. 

Finally, a Discounted Cash Flow (DCF) is calculated for each "actor" (project, CACER configuration, users).

Before to run the simulation be sure to have set correctly the following input files:
        
- input_FM.xlsx [[link excel](files/input_FM.xlsx)]

---

### **7. Calculate the electricity bills for each user types**

Before to run the simulation be sure to have set correctly the following input files:

        → files/mercato.yml

A csv file is created <u>for each actor</u> with the following data:
- **month**, in format [%Y-%M];

- **me_cost**, monthly cost for energy [€/month];
- **transport_cost**, monthly cost for transport [€/month];
- **ogs_cost**, monthly general system costs [€/month];
- **energy_cost**, monthly cost variable with electricity consumption [€/month];
- **fixed_cost**, monthly fixed cost [€/month];
- **power_cost**, monthly cost variable with contractual power [€/month];
- **duty_cost**, monthly cost for duty [€/month];
- **vat_cost**, monthly cost for vat [€/month];
- **total_bill_cost**, monthly total bill cost [€/month];
- **load_active**, monthly consumed energy [kWh/month];
- **load_active_corrected**, monthly consumed energy corrected with losses [kWh/month].

The csv files have the following structure:

| month  (type DatetimeIndex)| me_cost (type float)| transport_cost (type float)| ogs_cost (type float)| energy_cost (type float)| fixed_cost (type float)| power_cost (type float)| duty_cost (type float)| vat_cost (type float)| total_bill_cost (type float)| load_active (type float)| load_active_corrected (type float)|      
|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|:----:|
| 2025-01| 71.56| 71.56| 71.56| 71.56| 71.56| 71.56| 71.56| 71.56| 71.56| 71.56| 71.56|
| 2025-02| 58.89| 58.89| 58.89| 58.89| 58.89| 58.89| 58.89| 58.89| 58.89| 58.89| 58.89|
| ...| ...| ...| ...| ...| ...| ...| ...| ...| ...| ...| ...| 

📂 *files/finance/bills/*

&nbsp;

🚨 **PAY ATTENTION:**

- For consumers there are two sheets: one with the monthly values in BAU scenario (Business As Usual) and another one with the quarterly values.

- For prosumers and producers there are four sheets: monthly BAU scenario (without the photovoltaic generator), quarterly BAU scenario, monthly PV scenario (with the photovoltaic generator) and quarterly PV scenario.

- For project and for configuration (the xlsx file name is the same as the primary cabin name indicated in the *user CACER.xlsx* file) there are two sheets: monthly BAU scenario (without the photovoltaic generator) and monthly PV scenario (with the photovoltaic generator).

In [ ]:
print(blue("\n7. Create user bills:", ['bold', 'underlined']), '\n')
create_users_bill()

---

### **8. Setting name and type of the configuration and editing of the incentive repartition**

The "*case_denomination*" and the "*type_of_cacer*" are added to recap.yml file.

The "*case_denomination*" is used for the final report with results, for the folder with results, etc.

The "*type_of_cacer*" is necessary for the calculation of the incentives.

Finally, the incentive repartition between the members of the CACER configuration (internal and external) is setted in the following file:

        → files/inputs_FM.xlsx

&nbsp;

🚨 **PAY ATTENTION:**

The file *input_FM.xlsx* contains all the inputs for financila simulation.

It is possible to set the following repartition for the incentives:

* Egualitario
* Prosumer
* PNRR
* Equitario
* Misto CER
* Misto AUC
* Misto - ESCo
* Social Fund
* Misto - Social Fund
* Misto - Social Fund 2
* Misto - Social Fund PA
* Misto - PA
* CACER
        

In [ ]:
print(blue("\n8. Setting input financial simulation:", ['bold', 'underlined']), '\n')

case_denomination = "prova"

add_to_recap_yml(key = "case_denomination", value = case_denomination)
add_to_recap_yml(key = "type_of_cacer", value = "CER")
edit_incentive_repartition_scheme("Egualitario")
print("**** All inputs setted! ****")

---

### **9. Calculate CACER energy shared**

📘 **Formula**

The energy shared in the CACER configuration is calculated with the following formula:


$$ Hourly ~ energy ~ shared  = \min \left( \sum_{n_g=1}^{N_{gens}} ~ E_{inj, ~h, ~n_g}  ; \sum_{n_c=1}^{N_{consumers}} ~  E_{draw, ~h, ~n_c} \right) $$


🔍 **Meaning of the Terms**

- $
N_{gens}:~ \text{Number of energy generators (e.g., photovoltaic systems);}
$

- $
N_{consumers}:~ \text{Number of consumers in the community;}
$

- $
h:~ \text{Hour of the month;}
$

- $
E_{inj,h}:~ \text{Energy injected into the grid by a generator at hour } h;
$

- $
E_{draw,h}:~ \text{Energy drawn from the grid by a consumer at hour } h.
$



💾 **Outputs**

- In the csv file (*shared_energy_incentive_hourly.csv*) are collected the hourly values for the entire configuration and for each plants.

  The csv file has the following structure:

| dayhour  (type DatetimeIndex)| Eimm_CACER (type float)| Econd_config_ *Primary Station name* (type float)| Econd_ *user name* (type float)| ...| Econd_cond_CACER (type float)|
|:----:|:----:|:----:|:----:|:----:|:----:|
| 2025-01-01 00| 0| 0| 0| ...| 0|
| 2025-01-01 01| 0| 0| 0| ...| 0|
| ...| ...| ...| ...| ...| ...| 

📂 *files/energy/shared_energy_incentive_hourly.csv*

&nbsp;

- In the csv file (*shared_energy_incentive_yearly.csv*) are collected the yearly aggregated values for the entire configuration. 

  The csv file has the following structure:

| year  (type DatetimeIndex)| Eimm (type float)| Econd (type float)| perc_cond_annuale type float|
|:----:|:----:|:----:|:----:|
| 2025| 24512.54| 2133.47| 0.087|
| 2026| 24228.59| 2119.51| 0.087|
| ...| ...| ...| ...|

📂 *files/energy/shared_energy_incentive_hourly.csv*

&nbsp;

- In the csv file (*shared_energy_valorization_hourly.csv*) are collected the hourly values for the entire configuration and for each generators. The energy shared used for the calculation of the valorization can be different respect that calculated for the incentives (in this case also the "old" plants partecipate).

  The csv file has the following structure:

| dayhour  (type DatetimeIndex)| Econd_VAL (type float)| Econd_bt_VAL (type float)| Econd_mt_VAL (type float)| Econd_CP_VAL (type float)| Econd_CP_bt_VAL (type float)| Econd_CP_mt_VAL (type float)|
|:----:|:----:|:----:|:----:|:----:|:----:|:----:| 
| 2025-01-01 00| 0| 0| 0| 0| 0| 0|
| 2025-01-01 01| 0| 0| 0| 0| 0|  0|
| ...| ...| ...| ...| ...| ...| ...|

📂 *files/energy/shared_energy_valorization_hourly.csv*

In [ ]:
print(blue("\n9. Calculate CACER shared energy:", ['bold', 'underlined']))
CACER_shared_energy() # si calcola l'energia condivisa mensilmente dalla CER

---

### **10. Calculate energy sold revenues**

📘 **Formula**

The energy sold revenues of the generators are calculated with the following formula:


$$ Monthly ~ energy ~ sold ~ revenues = \sum_{n_g=1}^{N_{gens}} ~ \sum_{h=1}^{H_{month}} E_{inj, ~h, ~n_g} × PZO_h $$


🔍 **Meaning of the Terms**

- $
N_{gens}:~ \text{Number of energy generators (e.g., photovoltaic systems);}
$

- $
h:~ \text{Hour of the month;}
$

- $
H_{month}:~ \text{Total hours of the month;}
$

- $
E_{inj,h}:~ \text{Energy injected into the grid by a generator at hour } h;
$

- $
PZO_h:~ \text{Energy cost for the market zone at hour } h.
$



📝 **Inputs**

The inputs for the RID calculation are listed in:

        → files/RID/RID_input.yml

&nbsp;

💾 **Outputs**

- The first csv file (*output_RID.csv*) has the following structure:

| month  (type int)| *plant_id_1* (type float)| *plant_id_2* (type float)| ...|
|:----:|:----:|:----:|:----:|
| 2025-01| 107.32| 107.32| ...|
| 2025-02| 146.81| 146.81| ...| 
| ...| ...| ...| ...| 

📂 *files/RID/output_RID.csv*

&nbsp;

- In the csv file (*output_GSE_RID_fees.csv*) are collected the monthly fees cost for the RID accounting for each generators of the configuration.
  
  The csv file has the following structure:

| month  (type int)| *plants_id_1* (type float)| *plant_id_2* (type float)| ...|
|:----:|:----:|:----:|:----:|
| GSE_fees| 14| 14| ...|

📂 *files/RID/output_GSE_RID_fees.csv*

In [ ]:
print(blue("\n10. Calculate RID:", ['bold', 'underlined']))
RID_calculation()

---

### **11. Aggregate bills costs and energy sold revenues for the entire CACER configuration**


All the bills costs and energy sold revenues are aggregated for the entire configuration. In particular:

- The function **aggregate_CACER_bills()** create a csv file (*CACER_incentivi_per_configuration.csv*) with the following structure:

| month  (type DatetimeIndex)| incentivo (type float)| valorizzazione (type float)| incentivo_totale (type float)| surplus (type float)| social_fund (type float)|
|:----:|:----:|:----:|:----:|:----:|:----:|
| 2025-01| 17.61| 1.43| 17.61| 0| 0|
| 2025-02| 18.90| 1.53| 18.90| 0| 0|
| ...| ...| ...| ...| ...| ...| 

📂 *files/incentivi/CACER_incentivi_per_configuration.csv*

&nbsp;

- The function **aggregate_CACER_RID()** create a csv file (*output_RID_active_CACER.csv*) with the following structure:

| month  (type int)| *plant_id_1* (type float)| *plant_id_2* (type float)| ...| CER (type float)|
|:----:|:----:|:----:|:----:|:----:|
| 2025-01| 107.32| 107.32| ...| 1065.31|
| 2025-02| 146.81| 146.81| ...| 1854.54|
| ...| ...| ...| ...| ...| 

📂 *files/RID/output_RID_active_CACER.csv*

In [ ]:
print(blue("\n11. Aggregate bills and RID for the entire CACER:\n", ['bold', 'underlined']))
aggregate_CACER_bills()
aggregate_CACER_RID()

---

### **12. Generating the xlsx and csv files with all the financial inputs of the CACER configuration**

The following files are created:

        → files/finance/FM_template.csv

        → files/finance/investment_matrix.csv

        → files/finance/ownership_matrix.xlsx

        → files/finance/repartition_matrix.xlsx

        → files/finance/subscription_matrix.csv

        → files/finance/subscription_matrix.xlsx

        → files/finance/user_entry_matrix.csv

In [ ]:
print(blue("\n12. Initialize financial simulation:", ['bold', 'underlined']), '\n')
FM_initialization()

---

### **13. Calculate incentives revenues**

📘 **Formula incentives**

The incentives revenues of the configuration are calculated with the following formula:


$$ Monthly ~ incentives ~ revenues = \sum_{h=1}^{H_{month}} E_{shared, ~FIP, ~h} × FIP_h $$


🔍 **Meaning of the Terms**

- $
h:~ \text{Hour of the month;}
$

- $
H_{month}:~ \text{Total hours of the month;}
$

- $
E_{shared,FIP,h}:~ \text{Energy shared into the CACER configuration at hour } h;
$

- $
FIP_h:~ \text{Feed In Premium at hour } h.
$

&nbsp;

📘 **Formula valorization**

The valorization revenues of the configuration are calculated with the following formula:

$$ Monthly ~ valorization ~ revenues = CU_{Afa, m} × E_{ACV} + CU_{Afb), m} × E_{ACVC} + \sum_{i} (E_{ACV, ~i} × CP_{PR, ~i} × Pz) $$

🔍 **Meaning of the Terms**

- $
CU_{Afa, m}:~ \text{flat-rate monthly self-consumption unit fee }(TRAS_{E});
$

- $
E_{ACV}:~ \text{Monthly self-consumed energy;}
$

- $
CU_{Afb), m}:~ \text{flat-rate monthly self-consumption unit fee (BTAU)};
$

- $
E_{ACVC}:~ \text{Monthly self-consumed energy for voltage level i}
$

- $
E_{ACV, ~i}:~ \text{Hourly self-consumed energy for voltage level i}
$

- $
CP_{PR, ~i}:~ \text{coefficient of avoided losses (1.2\% for medium voltage and 2.6\% for low voltage)}
$

- $
Pz:~ \text{Hourly market price}
$

&nbsp;

💾 **Outputs**

The csv file (*CACER_incentivi.csv*) has the following structure:

| month_number  (type int)| incentivo (type float)| valorizzazione (type float)| incentivo_totale (type float)| surplus (type float)| social_fund (type float)|
|:----:|:----:|:----:|:----:|:----:|:----:|
| 1| 17.61| 1.43| 17.61| 0| 0|
| 2| 18.90| 1.53| 18.90| 0| 0|
| ...| ...| ...| ...| ...| ...| 

📂 *files/incentivi/CACER_incentivi.csv*

In [ ]:
print(blue("\n13. Calculate incentives:", ['bold', 'underlined']), '\n')
incentives()

---

### **14. All the economic cash flows for each users and for each stakeholders are calculated**

The function **cash_flows_for_all_plants()** calculate all the economic invoicse for the generators of the configuration.

A csv file (*plants_id.xlsx*) for each generators are created with the following sheets:

- Capex;

- D&A;

- Debt;

- Opex;

- Revenues;

- Totals.

📂 *files/finance/plants/*

&nbsp;

The function **cash_flows_for_all_users()** calculate all the economic cash flows for the users of the configuration, for the social fund and for the entire CACER configuration.

- A csv file (*user_id.xlsx*) for each generators are created with the following sheets:

    - plant_id;

    - CACER;

    - DCF_monthly;

    - DCF_yearly;

    - Results.

- A csv file (*social_fund.xlsx*) for each generators are created with the following economic cash flows:

    - month [%Y/%M];

    - inflaction_factor;
    
    - revenues_social_fund

- A csv file (*CACER.xlsx*) for each generators are created with the following sheets:

    - CACER;

    - totals;

    - DCF_monthly;

    - DCF_yearly;

    - Results.

📂 *files/finance/users/*

In [4]:
print(blue("\n14. Calculate cash flows for all power plants and for all users:", ['bold', 'underlined']), '\n')
cash_flows_for_all_plants() # plants first...
cash_flows_for_all_users() # ...then users...


14. Calculate cash flows for all power plants and for all users: 

CHECK: item= pv,	 size = 6,	 scale factor found: 1.35,	 capex €: 8910.00
CHECK: item= battery,	 size = 10,	 scale factor found: 1.15,	 capex €: 6900.00
CHECK: item= pv_inverter,	 size = 4.800000000000001,	 scale factor found: 1.20,	 capex €: 322.56
CHECK: item= mv_cabinet,	 size = 1,	 scale factor found: 0.00,	 capex €: 0.00
CHECK: item= battery,	 size = 10,	 scale factor found: 1.15,	 capex €: 4830.00
CHECK: item= pv,	 size = 20,	 scale factor found: 1.25,	 capex €: 27500.00
CHECK: item= pv_inverter,	 size = 16.0,	 scale factor found: 1.15,	 capex €: 1030.40
CHECK: item= mv_cabinet,	 size = 1,	 scale factor found: 0.00,	 capex €: 0.00
All files in files\\finance\\plants\\ folder deleted

Plant: p_D011 of 6kW and 10kWh
	- Capex pv:€ -8,910.00
	- Capex pv_inverter:€ -322.56
	- Capex battery:€ -6,900.00
	- Capex wind:€ -0.00
	- Capex mv_cabinet:€ -0.00
TOTAL CAPEX: € -16,132.56
Loan: € 0.00 at interest rate of 8.0% for 6

com_error: (-2147352567, 'Eccezione.', (0, 'Microsoft Excel', "Microsoft Excel cannot access the file 'C:\\Users\\rollo\\OneDrive - RSE S.p.A\\Desktop\\CACER - public repo\\CACER-simulator\\files\\inputs_FM.xlsx'. There are several possible reasons:\n\n• The file name or path does not exist.\n• The file is being used by another program.\n• The workbook you are trying to save has the same name as a currently open workbook.", 'xlmain11.chm', 0, -2146827284), None)

---

### **15. All the economic cash flows are aggregated for the project and for the CACER configuration (<u>for single primary station</u>)**

A csv file (*project.xlsx*) is created with the following sheets:

- plants;

- CACER;

- totals;

- DCF_monthly;

- DCF_yearly;

- Results.

&nbsp;

A csv file (*primary_station_id.xlsx*) for each primary stations are created with the following sheets:

- plants;

- CACER;

- totals;

- DCF_monthly;

- DCF_yearly;

- Results.

📂 *files/finance/configurations/*

In [ ]:
print(blue("\n15. Run financial model for each configurations:", ['bold', 'underlined']), '\n')
aggregate_FM() # ...then configurations!

---

### **16.1 The simulation results are organized for the export and for the creation of the report**

Different results are added to *files/recap.yml* and the following file is created:

        → files/results_finance/results_FM_last_simulation.csv

In [ ]:
print(blue("\n16.1 Organize results for report:", ['bold', 'underlined']), '\n')
organize_simulation_results_for_reporting()

---

### **16.2 Creating the report with results**

A report with the main results is created.

In [ ]:
print(blue("\n16.2 Create report:", ['bold', 'underlined']), '\n')
%run reporting_v3.ipynb

---

### **16.3 Creating the report with results**

All the following files and results are collected in a folder.

        → all files stored in files/finance/
        → files/energy/CACER_energy_monthly.xlsx
        → files/incentivi/CACER_incentivi.csv
        → files/membership_matrix.csv
        → files/plant_operation_matrix.xlsx
        → files/inputs_FM.xlsx
        → files/registry_plants.yml
        → files/registry_user_types.yml
        → files/registry_users.yml
        → files/results_finance/results_FM_last_simulation.csv
        → files/recap.yml
        → the report stored in files/results_finance/case_denomination_id.docx

📂 *files/results_finance/case_denomination/*

In [ ]:
print(blue("\n16.3 Save all finance results:", ['bold', 'underlined']), '\n')
save_simulation_results(simulation_name = case_denomination)

---
---